In [ ]:
import json
import requests
import time
import folium
from kaggle_secrets import UserSecretsClient

In [ ]:
exec(open("/kaggle/input/llm-20-questions/llm_20_questions/keywords.py").read())
KEYWORDS_JSON = json.loads(KEYWORDS_JSON)
KEYWORDS_JSON

In [ ]:
print(len(KEYWORDS_JSON))
for category in KEYWORDS_JSON:
    print(category["category"], len(category["words"]))

In [ ]:
user_secrets = UserSecretsClient()
geocoding_api_key = user_secrets.get_secret("geocoding_api_key") # add your api_key in 'Add-ons'->'secret' (see https://www.kaggle.com/discussions/product-feedback/114053)

In [ ]:
url_api = "https://geocode.maps.co/search?q={address}&api_key={api_key}"
# test api
r = requests.get(url_api.format(address="croatia",api_key=geocoding_api_key))
r.status_code

In [ ]:
r.json()

In [ ]:
worldmap = folium.Map([0, 0], titles='Map', zoom_start=4)

In [ ]:
colors = {"country":"blue","city":"green","landmark":"orange"}
for category, name in zip(KEYWORDS_JSON,["country","city","landmark"]):
    for location in category["words"]:
        r = requests.get(url_api.format(address=location["keyword"],api_key=geocoding_api_key))
        if r.status_code != 200:
            print(f"error {r.status_code = } {name} {location['keyword']}")
            continue
        try:
            resp = r.json()[0]
            folium.Circle(location = [resp.get('lat'),resp.get('lon')],
                      popup=f"{name} {location['keyword']}",
                      color=colors[name],
                      tooltip=f"{name} {location['keyword']} {location['alts']}",
                      fill = False).add_to(worldmap)
        except IndexError:
            print("error empty response", name, location["keyword"])
        time.sleep(1) # limit of 1 request per second for free plan
    
worldmap